# Q/A with OpenAI and Trino

## Define Database Connection
This is utilizing an AirBnB data set from the [toolkit use case](https://github.com/msantana09/data-engineering-toolkit/blob/main/UseCase.md)

In [19]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine

URI = 'trino://trino@localhost:8082/lakehouse/kaggle_airbnb'
OPENAI_MODEL = "gpt-3.5-turbo"

engine = create_engine(URI)
db =SQLDatabase(engine=engine )


/Users/miguel/Documents/jobs/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'record_count' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/Documents/jobs/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'file_count' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/Documents/jobs/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'total_size' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/Documents/jobs/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'data' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/Documents/jobs/fun_with_llms/.venv/lib/python3

### Define LLM and SQL Agent

In [20]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
sql_agent = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

### Test it out

In [21]:
response = sql_agent.invoke(
    "Which neighborhood has the best reviews?"
)
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


listings, reviews
Invoking: `sql_db_schema` with `reviews`



CREATE TABLE reviews (
	listing_id INTEGER, 
	id INTEGER, 
	date DATE, 
	reviewer_id INTEGER, 
	reviewer_name VARCHAR, 
	comments VARCHAR
)

/*
3 rows from reviews table:
listing_id	id	date	reviewer_id	reviewer_name	comments
7202016	38917982	2015-07-19	28943674	Bianca	Cute and cozy place. Perfect location to everything!
7202016	39087409	2015-07-20	32440555	Frank	Kelly has a great room in a very central location. 
Beautiful building , architecture and a style t
7202016	39820030	2015-07-26	37722850	Ian	Very spacious apartment, and in a great neighborhood.  This is the kind of apartment I wish I had!

*/
Invoking: `sql_db_query` with `SELECT listing_id, AVG(rating) AS avg_rating FROM reviews GROUP BY listing_id ORDER BY avg_rating DESC LIMIT 10`


Error: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=COLUMN_NOT_FOUND, mes